# Dataset
This project uses the [MedleyDB dataset](https://medleydb.weebly.com/) V1 and V2. To get access to the dataset, you need to ask for permission through Zenodo.

This notebook is created to explore the MedleyDB dataset. There is an overview of the dataset here: https://medleydb.weebly.com/description.html

In [2]:
%load_ext autoreload
%autoreload 2

In [54]:
import numpy as np
import pandas as pd
import os
import random

If import medleydb return `TypeError: load() missing 1 required positional argument: 'Loader'`, you need to downgrade pyyamel:
```
pyyaml==5.4.1
```

In [ ]:
# path to a forked medleydb repo https://github.com/duotien/medleydb
medleydb_repo_path = "/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/"
medleydb_data_path = os.path.join(medleydb_repo_path, 'medleydb/data/')
print(medleydb_data_path)
os.environ['MEDLEYDB_PATH'] = medleydb_data_path
import medleydb
from pitch_tracker.utils.medleydb_melody import gen_label

# Generate annotations
medleyDB also has tools for customizing the annotations. However you need to have the audio for it to work.

In [ ]:
HOP = 512*5
output_dir = f'../content/gen_label/{HOP}'
mtracks = medleydb.load_all_multitracks()

for mtrack in mtracks:
    gen_label(mtrack.track_id, output_dir, hop=HOP, overwrite=True, convert_to_midi=True, round_method='round', to_csv=True)
    # gen_label(mtrack.track_id, output_dir, hop=HOP, overwrite=True, convert_to_midi=True, round_method='round', to_csv=False)

In [32]:
melody_1_path, melody_2_path, *_ = [os.path.join(output_dir, folder_path) for folder_path in os.listdir(output_dir)]


In [55]:
melody_2_annotation_paths = []
for root, folders, files in os.walk(melody_2_path):
    melody_2_annotation_paths.extend([os.path.join(root, fpath) for fpath in files])

In [63]:
sample_annotation_path = random.choice(melody_2_annotation_paths)
print(sample_annotation_path)
df = pd.read_csv(sample_annotation_path, names=['start', 'end', 'midi_value'])
df.head(5)

../content/gen_label/2560/Melody2_midi/Auctioneer_OurFutureFaces.csv


,start,end,midi_value
0,1.277098,1.335147,57.0
1,1.335147,1.451247,58.0
2,1.451247,1.509297,59.0
3,1.509297,1.567347,58.0
4,1.625397,1.915646,59.0


# Visualise
We recommend using Sonic Visualiser to visualize the annotations with audios.

In [64]:
# code to visualize using matplotlib
from matplotlib import pyplot as plt